### Step 1: Data generation and training

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from PIL import Image, UnidentifiedImageError
from tqdm.notebook import tqdm
import signal
from concurrent.futures import ThreadPoolExecutor, as_completed, TimeoutError
import glob
import logging

In [2]:
# Set the directory for saving checkpoints and logs
save_dir = '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Ensemble Model/Step 1 outputs'
os.makedirs(save_dir, exist_ok=True)

# Directories
input_dir1_original = '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Generated Images/ma-boston/buildings'
input_dir1_target = '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Generated Images/ma-boston/parcels'
output_dir1 = '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Trained Models/ma-boston-p2p-500-150-v100/web-boston/images'
input_dir2_original = '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Generated Images/nc-charlotte/buildings'
input_dir2_target = '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Generated Images/nc-charlotte/parcels'
output_dir2 = '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Trained Models/nc-charlotte-500-150-v100/web-charlotte/images'
input_dir3_original = '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Generated Images/ny-manhattan/buildings'
input_dir3_target = '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Generated Images/ny-manhattan/parcels'
output_dir3 = '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Trained Models/ny-manhattan-p2p-500-150-v100/web-manhattan/images'
input_dir4_original = '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Generated Images/pa-pittsburgh/buildings'
input_dir4_target = '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Generated Images/pa-pittsburgh/parcels'
output_dir4 = '/Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawing Participation/Million Neighborhoods/Trained Models/pa-pittsburgh-p2p-500-150-v100/web-pittsburgh/images'

In [3]:
# Setup logging
log_file = os.path.join(save_dir, 'image_loading.log')
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define a custom dataset class
class ImageDataset(Dataset):
    def __init__(self, original_images, target_images, output_images, labels, transform=None):
        self.original_images = original_images
        self.target_images = target_images
        self.output_images = output_images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        original_image = self.original_images[idx]
        target_image = self.target_images[idx]
        output_image = self.output_images[idx]
        label = self.labels[idx]
        if self.transform:
            original_image = self.transform(original_image)
            target_image = self.transform(target_image)
            output_image = self.transform(output_image)
        return original_image, target_image, output_image, label

# Define a timeout handler
def handler(signum, frame):
    raise TimeoutError("Image loading timed out")

# Set the signal handler and a 20-second alarm
signal.signal(signal.SIGALRM, handler)

# Function to load a single image with retry mechanism
def load_single_image(img_path, transform, retries=3):
    attempt = 0
    while attempt < retries:
        try:
            signal.alarm(20)  # Trigger alarm in 20 seconds
            img = Image.open(img_path).convert('RGB')
            img = transform(img)
            signal.alarm(0)  # Disable the alarm
            return img
        except (UnidentifiedImageError, TimeoutError, Exception) as e:
            logging.warning(f"Cannot process image file {img_path}, attempt {attempt+1}/{retries}. Error: {e}")
            signal.alarm(0)  # Disable the alarm in case of an exception
            attempt += 1
    logging.error(f"Failed to load image {img_path} after {retries} attempts, skipping.")
    return None

# Function to remove old checkpoints, keeping only the latest two
def remove_old_checkpoints(checkpoint_pattern):
    checkpoints = sorted(glob.glob(checkpoint_pattern), key=os.path.getmtime)
    while len(checkpoints) > 2:
        old_checkpoint = checkpoints.pop(0)
        os.remove(old_checkpoint)
        logging.info(f"Removed old checkpoint: {old_checkpoint}")

# Function to load images from a directory, filtering by a keyword, with parallel processing
def load_images_from_directory(directory, keyword=None, target_size=(256, 256), max_workers=8):
    transform = transforms.Compose([
        transforms.Resize(target_size),
        transforms.ToTensor()
    ])
    images = []
    filenames = [f for f in sorted(os.listdir(directory)) if keyword is None or keyword in f]
    img_paths = [os.path.join(directory, f) for f in filenames]
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_path = {executor.submit(load_single_image, path, transform): path for path in img_paths}
        for i, future in enumerate(tqdm(as_completed(future_to_path, timeout=30), total=len(img_paths), desc=f"Loading images from {directory}")):
            try:
                img = future.result()
                if img is not None:
                    images.append(img)
                if (i + 1) % 250 == 0:  # Save progress every 250 images
                    checkpoint_file = os.path.join(save_dir, f'checkpoint_{directory.replace("/", "_")}_{i+1}.pt')
                    torch.save(images, checkpoint_file)
                    logging.info(f"Checkpoint saved: {checkpoint_file}")
                    # Remove old checkpoints
                    remove_old_checkpoints(os.path.join(save_dir, f'checkpoint_{directory.replace("/", "_")}*.pt'))
            except TimeoutError:
                logging.error(f"Timeout error for image {future_to_path[future]}")
            except Exception as e:
                logging.error(f"Error processing image {future_to_path[future]}: {e}")
    
    return torch.stack(images) if images else torch.empty((0, 3, target_size[0], target_size[1]))

# Load images with checkpointing
def load_images_with_checkpointing(input_dir, keyword, checkpoint_file, target_size=(256, 256), max_workers=8):
    if os.path.exists(checkpoint_file):
        print(f"Loading from checkpoint: {checkpoint_file}")
        loaded_images = torch.load(checkpoint_file)
    else:
        loaded_images = []
    
    def save_checkpoint(images, index):
        torch.save(images, checkpoint_file)
        checkpoint_pattern = os.path.join(save_dir, f'checkpoint_{os.path.basename(checkpoint_file)}_{index}.pt')
        torch.save(images, checkpoint_pattern)
        remove_old_checkpoints(os.path.join(save_dir, f'checkpoint_{os.path.basename(checkpoint_file)}*.pt'))
    
    transform = transforms.Compose([
        transforms.Resize(target_size),
        transforms.ToTensor()
    ])
    
    filenames = [f for f in sorted(os.listdir(input_dir)) if keyword is None or keyword in f]
    img_paths = [os.path.join(input_dir, f) for f in filenames][len(loaded_images):]  # Skip already loaded images
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_path = {executor.submit(load_single_image, path, transform): path for path in img_paths}
        for i, future in enumerate(tqdm(as_completed(future_to_path), total=len(img_paths), desc=f"Loading images from {input_dir}")):
            try:
                img = future.result()
                if img is not None:
                    loaded_images.append(img)
                if (i + 1) % 250 == 0:  # Save progress every 250 images
                    save_checkpoint(loaded_images, i + 1)
            except Exception as e:
                logging.error(f"Error processing image {future_to_path[future]}: {e}")
    
    save_checkpoint(loaded_images, 'final')  # Final save
    return torch.stack(loaded_images) if loaded_images else torch.empty((0, 3, target_size[0], target_size[1]))

# Load original and target input images with checkpointing
input_images_model1_original = load_images_with_checkpointing(input_dir1_original, '', os.path.join(save_dir, 'checkpoint_model1_original.pt'))
input_images_model1_target = load_images_with_checkpointing(input_dir1_target, '', os.path.join(save_dir, 'checkpoint_model1_target.pt'))
input_images_model2_original = load_images_with_checkpointing(input_dir2_original, '', os.path.join(save_dir, 'checkpoint_model2_original.pt'))
input_images_model2_target = load_images_with_checkpointing(input_dir2_target, '', os.path.join(save_dir, 'checkpoint_model2_target.pt'))
input_images_model3_original = load_images_with_checkpointing(input_dir3_original, '', os.path.join(save_dir, 'checkpoint_model3_original.pt'))
input_images_model3_target = load_images_with_checkpointing(input_dir3_target, '', os.path.join(save_dir, 'checkpoint_model3_target.pt'))
input_images_model4_original = load_images_with_checkpointing(input_dir4_original, '', os.path.join(save_dir, 'checkpoint_model4_original.pt'))
input_images_model4_target = load_images_with_checkpointing(input_dir4_target, '', os.path.join(save_dir, 'checkpoint_model4_target.pt'))

# Load only the fake_B output images with checkpointing
output_images_model1 = load_images_with_checkpointing(output_dir1, 'fake_B', os.path.join(save_dir, 'checkpoint_model1_fake_B.pt'))
output_images_model2 = load_images_with_checkpointing(output_dir2, 'fake_B', os.path.join(save_dir, 'checkpoint_model2_fake_B.pt'))
output_images_model3 = load_images_with_checkpointing(output_dir3, 'fake_B', os.path.join(save_dir, 'checkpoint_model3_fake_B.pt'))
output_images_model4 = load_images_with_checkpointing(output_dir4, 'fake_B', os.path.join(save_dir, 'checkpoint_model4_fake_B.pt'))

# Create dataset and labels
original_images = []
target_images = []
output_images_all = []
labels = []

# Using a progress bar for the image and label appending loop
for i in tqdm(range(len(input_images_model1_original)), desc="Combining images and labels"):
    original_images.append(input_images_model1_original[i])
    target_images.append(input_images_model1_target[i])
    output_images_all.append(output_images_model1[i])
    labels.append(0)  # Label for model 1
    original_images.append(input_images_model2_original[i])
    target_images.append(input_images_model2_target[i])
    output_images_all.append(output_images_model2[i])
    labels.append(1)  # Label for model 2
    original_images.append(input_images_model3_original[i])
    target_images.append(input_images_model3_target[i])
    output_images_all.append(output_images_model3[i])
    labels.append(2)  # Label for model 3
    original_images.append(input_images_model4_original[i])
    target_images.append(input_images_model4_target[i])
    output_images_all.append(output_images_model4[i])
    labels.append(3)  # Label for model 4

# Convert dataset and labels to tensors
original_images = torch.stack(original_images)
target_images = torch.stack(target_images)
output_images_all = torch.stack(output_images_all)
labels = torch.tensor(labels)

# Save datasets to disk
torch.save({
    'original_images': original_images,
    'target_images': target_images,
    'output_images_all': output_images_all,
    'labels': labels
}, os.path.join(save_dir, 'dataset.pt'))

# Split dataset according to specifications
train_original = original_images[:20000]
train_target = target_images[:20000]
train_output = output_images_all[:20000]
train_labels = labels[:20000]

buffer_original = original_images[20000:22500]
buffer_target = target_images[20000:22500]
buffer_output = output_images_all[20000:22500]
buffer_labels = labels[20000:22500]

test_original = original_images[22500:23750]
test_target = target_images[22500:23750]
test_output = output_images_all[22500:23750]
test_labels = labels[22500:23750]

val_original = original_images[23750:25000]
val_target = target_images[23750:25000]
val_output = output_images_all[23750:25000]
val_labels = labels[23750:25000]

# Save splits to disk
torch.save({
    'train_original': train_original,
    'train_target': train_target,
    'train_output': train_output,
    'train_labels': train_labels,
    'buffer_original': buffer_original,
    'buffer_target': buffer_target,
    'buffer_output': buffer_output,
    'buffer_labels': buffer_labels,
    'test_original': test_original,
    'test_target': test_target,
    'test_output': test_output,
    'test_labels': test_labels,
    'val_original': val_original,
    'val_target': val_target,
    'val_output': val_output,
    'val_labels': val_labels
}, os.path.join(save_dir, 'dataset_splits.pt'))

# Create DataLoader objects
batch_size = 32 # 16, 32, 64…
train_dataset = ImageDataset(train_original, train_target, train_output, train_labels, transform=transforms.ToTensor())
test_dataset = ImageDataset(test_original, test_target, test_output, test_labels, transform=transforms.ToTensor())
val_dataset = ImageDataset(val_original, val_target, val_output, val_labels, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

Loading images from /Users/ls/Library/CloudStorage/GoogleDrive-l.schrage@northeastern.edu/Shared drives/Drawin…

TimeoutError: Image loading timed out

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/homebrew/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/homebrew/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/homebrew/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/opt/homebrew/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/opt/homebrew/Cellar/python@3.11/3.11.9_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
    self._run_once()
  File "/opt/homebrew/Cellar/python@3.11/3.11.9_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.py", line 1898, in _run_once
    event_lis

: 

In [ ]:
# Define the CNN model
class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 64 * 64, 128)  # Adjust according to the input image size
        self.fc2 = nn.Linear(128, 4)  # 4 classes for 4 models
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 64 * 64)  # Adjust according to the input image size
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.softmax(x)
        return x

# Initialize the model, loss function, and optimizer
model = CNNClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')

### Step 2: Auto-encoder training

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

In [ ]:
# Define the AutoEncoder model
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2)
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, kernel_size=2, stride=2),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Initialize the autoencoder model
autoencoder = AutoEncoder()

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

In [ ]:
# Define a custom dataset class for the autoencoder
class AutoEncoderDataset(Dataset):
    def __init__(self, images, transform=None):
        self.images = images
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        if self.transform:
            image = self.transform(image)
        return image, image

# Create DataLoader objects for the autoencoder training
transform = transforms.ToTensor()
autoencoder_dataset = AutoEncoderDataset(output_images_model1 + output_images_model2 + output_images_model3 + output_images_model4, transform=transform)
autoencoder_loader = DataLoader(autoencoder_dataset, batch_size=32, shuffle=True)

In [ ]:
# Train the autoencoder
num_epochs = 50
for epoch in range(num_epochs):
    autoencoder.train()
    running_loss = 0.0
    for images, _ in autoencoder_loader:
        optimizer.zero_grad()
        outputs = autoencoder(images)
        loss = criterion(outputs, images)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(autoencoder_loader):.4f}')

# Save the trained autoencoder model
torch.save(autoencoder.state_dict(), 'autoencoder.pth')

### Step 3: Combining latent vectors and reconstruction

In [ ]:
import torch.nn.functional as F

# Load the trained classifier model
classifier = CNNClassifier()
classifier.load_state_dict(torch.load('classifier.pth'))
classifier.eval()

# Load the trained autoencoder model
autoencoder = AutoEncoder()
autoencoder.load_state_dict(torch.load('autoencoder.pth'))
autoencoder.eval()

# Define the encoder and decoder separately for easier handling
encoder = autoencoder.encoder
decoder = autoencoder.decoder

In [ ]:
# Function to combine latent vectors with weights
def combine_latent_vectors(vectors, weights):
    combined_vector = sum(w * v for w, v in zip(weights, vectors))
    return combined_vector

# Assume you have an input image to process
input_image_path = '/path/to/input/image.png'
input_image = Image.open(input_image_path).resize((256, 256))
input_image = transforms.ToTensor()(input_image).unsqueeze(0)  # Convert to tensor and add batch dimension

# Get the output images from the four individual models
output1 = model1(input_image)
output2 = model2(input_image)
output3 = model3(input_image)
output4 = model4(input_image)

# Pass these output images through the encoder part of the auto-encoder to get four latent vectors
latent_vector1 = encoder(output1).detach()
latent_vector2 = encoder(output2).detach()
latent_vector3 = encoder(output3).detach()
latent_vector4 = encoder(output4).detach()

# Use the classifier to get the softmax weights for the four latent vectors
with torch.no_grad():
    weights = classifier(input_image).softmax(dim=1).squeeze()

# Combine these latent vectors using the weights to get a single combined latent vector
combined_vector = combine_latent_vectors([latent_vector1, latent_vector2, latent_vector3, latent_vector4], weights)

# Pass the combined latent vector through the decoder part of the auto-encoder to get the final output image
final_output_image = decoder(combined_vector.unsqueeze(0))

# Convert the output tensor to an image format and save it
final_output_image = final_output_image.squeeze().permute(1, 2, 0).numpy()
final_output_image = (final_output_image * 255).astype(np.uint8)
save_path = '/path/to/output/image.png'
Image.fromarray(final_output_image).save(save_path)